In [25]:
import induceC45
import classify
import pandas as pd
import numpy as np

In [43]:
raw = pd.read_csv("nursery.csv", skiprows=[1], header=0)
classvar = raw.iloc[0, 0]
nursery = raw.drop(0, axis=0)
nursery

,parents,has_nurs,form,children,housing,finance,social,health,class
1,usual,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
2,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
3,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
4,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
5,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority
...,...,...,...,...,...,...,...,...,...
12956,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,priority,spec_prior
12957,great_pret,very_crit,foster,more,critical,inconv,slightly_prob,not_recom,not_recom
12958,great_pret,very_crit,foster,more,critical,inconv,problematic,recommended,spec_prior
12959,great_pret,very_crit,foster,more,critical,inconv,problematic,priority,spec_prior


In [13]:
nursery_split = np.array_split(nursery.sample(frac=1), 5)
type(nursery_split)

C:\Users\hsu_m\anaconda3\envs\csc466\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


list

In [18]:
def split_folds(df: pd.DataFrame, k: int) -> list:
    return np.array_split(df.sample(frac=1), k)

In [37]:
nursery_split = split_folds(nursery, 5)
nursery_split

C:\Users\hsu_m\anaconda3\envs\csc466\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


[           parents     has_nurs        form children     housing     finance  \
 882          usual  less_proper    complete        1  convenient      inconv   
 2133         usual     improper   completed     more   less_conv  convenient   
 12404   great_pret    very_crit   completed        2    critical  convenient   
 3184         usual     critical  incomplete        3    critical      inconv   
 11772   great_pret     critical  incomplete        2    critical      inconv   
 ...            ...          ...         ...      ...         ...         ...   
 8257   pretentious    very_crit  incomplete        1    critical      inconv   
 3852         usual    very_crit   completed     more  convenient      inconv   
 8542   pretentious    very_crit      foster        3  convenient      inconv   
 3609         usual    very_crit    complete        3    critical  convenient   
 11185   great_pret     improper      foster     more  convenient  convenient   
 
               social     

In [38]:
threshold = 0.7
gratio = False

preds = []
results = []
for i in range(len(nursery_split)):
    # Take fold i as training data, fit tree.
    fold = nursery_split[i]
    C_train = fold[classvar]
    D_train = fold.drop(classvar, axis=1)
    A_train = list(D_train.columns)

    T = induceC45.C45(D_train, A_train, C_train, threshold, gratio=gratio)

    # Take rest of folds, combine, and test.
    test = nursery_split[:]
    del test[i]
    test = pd.concat(test)

    pred = classify.predict(test, T)
    preds.append(pred)
    results.append(classify.report(pred, classvar, 'pred'))

preds = pd.concat(preds)
results = pd.DataFrame(results)
overall_cv_report = classify.report(preds, classvar, 'pred')
overall_cv_report['avg_accuracy'] = results['accuracy'].mean()
overall_cv_report

{'df':            parents   has_nurs        form children     housing     finance  \
 3611         usual  very_crit    complete        3    critical      inconv   
 365          usual     proper   completed        3    critical  convenient   
 4559   pretentious     proper   completed        1   less_conv  convenient   
 10672   great_pret   improper   completed        2   less_conv      inconv   
 6387   pretentious   improper   completed        3  convenient      inconv   
 ...            ...        ...         ...      ...         ...         ...   
 11834   great_pret   critical  incomplete     more  convenient  convenient   
 8142   pretentious  very_crit   completed        3    critical  convenient   
 12761   great_pret  very_crit      foster        1  convenient      inconv   
 6292   pretentious   improper   completed        1   less_conv      inconv   
 3002         usual   critical   completed     more   less_conv      inconv   
 
               social       health       cla

In [39]:
def cv_report(splits, classvar, threshold, gratio=False):
    preds = []
    results = []

    for i in range(len(splits)):
        # Take fold i as training data, fit tree.
        fold = splits[i]
        C_train = fold[classvar]
        D_train = fold.drop(classvar, axis=1)
        A_train = list(D_train.columns)

        T = induceC45.C45(D_train, A_train, C_train, threshold, gratio=gratio)

        # Take rest of folds, combine, and test.
        test = splits[:]
        del test[i]
        test = pd.concat(test)

        pred = classify.predict(test, T)
        preds.append(pred)
        results.append(classify.report(pred, classvar, 'pred'))

    preds = pd.concat(preds)
    results = pd.DataFrame(results)
    cv_report = classify.report(preds, classvar, 'pred')
    cv_report['avg_accuracy'] = results['accuracy'].mean()
    return cv_report

In [40]:
cv_report(nursery_split, classvar, 0.7)

{'df':            parents   has_nurs        form children     housing     finance  \
 3611         usual  very_crit    complete        3    critical      inconv   
 365          usual     proper   completed        3    critical  convenient   
 4559   pretentious     proper   completed        1   less_conv  convenient   
 10672   great_pret   improper   completed        2   less_conv      inconv   
 6387   pretentious   improper   completed        3  convenient      inconv   
 ...            ...        ...         ...      ...         ...         ...   
 11834   great_pret   critical  incomplete     more  convenient  convenient   
 8142   pretentious  very_crit   completed        3    critical  convenient   
 12761   great_pret  very_crit      foster        1  convenient      inconv   
 6292   pretentious   improper   completed        1   less_conv      inconv   
 3002         usual   critical   completed     more   less_conv      inconv   
 
               social       health       cla

In [ ]:
for row in nursery.iterrows():
    # Take fold i as training data, fit tree.
    fold = nursery_split[i]
    C_train = fold[classvar]
    D_train = fold.drop(classvar, axis=1)
    A_train = list(D_train.columns)

    T = induceC45.C45(D_train, A_train, C_train, threshold, gratio=gratio)

In [97]:
def fit_loo(i: int, df: pd.DataFrame, classvar: str, threshold: float, gratio: bool = False):
    loo = df.drop(i, axis=0)
    C = loo[classvar]
    D = loo.drop(classvar, axis=1)
    A = list(D.columns)

    T = induceC45.C45(D, A, C, threshold, gratio=gratio)

    row = df.loc[i].to_frame().T
    pred = classify.predict(row, T)['pred'].loc[i]
    return pred

In [98]:
fit_loo(5, nursery, classvar, 0.7, False)

'spec_prior'

In [99]:
pd.Series(nursery.index).sample(n=10).apply(lambda i: fit_loo(i, nursery, classvar, 0.7, False))

3959      not_recom
91       spec_prior
10673     not_recom
9640     spec_prior
9371      not_recom
4943      not_recom
5040       priority
3575      not_recom
5876      not_recom
9819       priority
dtype: object

In [ ]:
def loo_predict(df: pd.DataFrame, classvar: str, threshold: float, gratio: bool = False):
    pred = pd.Series(df.index).apply(lambda i: fit_loo(i, df, classvar, threshold, gratio))
    df['pred'] = pred
    return df